In [1]:
import os
import requests
import getpass
import datetime
import pytz
from requests.auth import HTTPBasicAuth
from pathlib import Path

import pandas as pd
import geopandas as gpd
import shapely

from ipyleaflet import GeoJSON
from planet import api
from traitlets import Unicode, link
import ipyvuetify as v

from sepal_ui import sepalwidgets as sw
from sepal_ui import mapping as m;
from sepal_ui.scripts import utils as su

HTML(value='\n<style>\n.leaflet-pane {\n    z-index : 2 !important;\n}\n.leaflet-top, .leaflet-bottom {\n    z…

ResizeTrigger()

In [2]:
def hide_password(widget, event, data):
    
    if widget.type=='text':
        widget.type='password'
        widget.append_icon = 'mdi-eye-off'
    else:
        widget.type = 'text'
        widget.append_icon = 'mdi-eye'

In [3]:
api_key = "208fcdc9f18e46999211cda26ae4b1bc" # NICFI Vicente

In [4]:
api_key = "e74724542185443697ad280f680859d0" # Daniel

In [5]:
class PlanetKey:
    
    def __init__(self, api_key):
        
        self.api_key = api_key
        self.url = 'https://api.planet.com/auth/v1/experimental/public/my/subscriptions'
        self.subs = None
        self.active = None
        
    def get_subscription(self):
        
        resp = requests.get(self.url, auth=(self.api_key, ''))
        subscriptions = resp.json()
        
        if resp.status_code == 200: return subscriptions
    
    def is_active(self):
        
        subs = self.get_subscription()
        active = [False]
        
        if subs: active = [True for sub in subs if sub['state'] == 'active']
        
        return any(active)


In [6]:
msg = {
    'fill_api': 'Please use a Planet API key and validate it.',
    'success_api': ('Your API key is valid', 'success'),
    'fail_api': ('Your API key is not valid', 'error')
}

In [7]:
COUNTRIES = gpd.read_file('https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json')

In [8]:
class UI(v.Layout):
    
    USER = getpass.getuser()
    
    timespan = Unicode('24 hours').tag(sync=True)
    
    def __init__(self, **kwargs):
        
        self.class_='pa-2'

        
        super().__init__(**kwargs)
        
        # Start workspace
        self.root_dir, self.data_dir = self._workspace()
        
        self.alerts = None
        self.aoi = None
        self.aoi_alerts = None
        
        
        self.map_ = m.SepalMap(basemaps=['Google Satellite'], dc=True)
        self.map_.show_dc()
        
        # Widgets
        
        self.w_api_key = v.TextField(
            label="Planet API Key",
            class_='mr-2',
            v_model='208fcdc9f18e46999211cda26ae4b1bc',
            type='password', 
            append_icon='mdi-eye-off'
        )
        self.w_api_btn = sw.Btn('Check ', small=True,)
        
        self.w_spantime = v.Select(
            label="In the last",
            items=['24 hours', '48 hours', '7 days'],
            v_model=self.timespan,
        )
        
        self.w_aoi_method = v.Select(
            label="AOI method",
            v_model='Draw on map',
            items=['Draw on map', 'Select country'],
            
        )
        self.w_countries = v.Select(
            label="Select country",
            v_model='',
            items=COUNTRIES.name.to_list(),
        )
        
        self.w_alert = sw.Alert()
        
        self.w_prev = sw.Btn('prev', 
                             x_small=True, 
                             color="secondary")
        
        self.w_next = sw.Btn('nxt', 
                             x_small=True, 
                             color="secondary")
        
        self.w_alert_list = v.Select(
            class_='ma-2',
            label='Alert', 
            items=[],
            v_model=None
        )
        
        self.w_alerts = v.Card(
            class_='pl-2 pr-2', 
            children=[
                v.Flex(
                class_='d-flex align-center mb-2', row=True,
                children=[
                    self.w_prev,
                    self.w_alert_list,
                    self.w_next
                ],
            )], 
            disabled=True)
        
        self.w_run = sw.Btn("Get Alerts")
        
        self.w_api_alert = sw.Alert()
        

        su.hide_component(self.w_countries)
        
        
        # Events
        
        self.w_countries.observe(self.add_country_event, 'v_model')
        self.w_aoi_method.observe(self.aoi_method_event, 'v_model')
        self.w_alert_list.observe(self.alert_list_event, 'v_model')
        
        self.w_api_btn.on_event('click', self.validate_api_event)
        self.w_prev.on_event('click', self.prev_next_event)
        self.w_next.on_event('click', self.prev_next_event)
        self.w_api_key.on_event('click:append', hide_password)
        
        self.map_.dc.on_draw(self.handle_draw)
        self.w_run.on_event('click', self._get_alerts)
        
        
        link((self.w_spantime, 'v_model'),(self, 'timespan'))
        

        # View
        
        self.opt_panel = v.Card(
            class_='pa-2',
            children=[
                self.w_spantime,
                self.w_aoi_method,
                self.w_countries,
                self.w_run,
                self.w_alert,
            ],
            disabled = True)
        
        self.children = (
            
            v.Flex(xs12 = True, children =[
                v.Card(class_='pa-2 mb-2', children=[
                    v.Flex(class_='d-flex align-center mb-2', row=True, children =[self.w_api_key, self.w_api_btn]),
                    self.w_api_alert,                    
                ]),
                self.opt_panel,
            ]),
            
            # Right flex for map
            v.Flex(class_='ml-2', xs12 = True, children =[
                self.w_alerts,
                self.map_
            ])
        )
        
    def validate_api_event(self, widget, change, data):
        
        api_key = self.w_api_key.v_model
        
        planet_key = PlanetKey(api_key)
        
        valid = planet_key.is_active()
        
        if valid:
            self.w_api_alert.add_msg(msg['success_api'][0], msg['success_api'][1])
            self.opt_panel.disabled=False
        else:
            self.w_api_alert.add_msg(msg['fail_api'][0], msg['fail_api'][1])
            self.opt_panel.disabled=True
            
    def remove_layers(self):
        
        # get map layers
        layers = self.map_.layers
        
        # loop and remove layers 
        [self.map_.remove_last_layer() for _ in range(len(layers))]
        
        
    def handle_draw(self, target, action, geo_json):
        
        self.remove_layers()
        if action == 'created':
            self.aoi = geo_json['geometry']
    
    def alert_list_event(self, change):
        """ Update map zoom and center when selecting an alert
        
        """
        
        # Get fire alert id
        
        alert_id = change['new']
        
        # Filter dataframe to get lat,lon
        
        lat = self.aoi_alerts.loc[alert_id]['latitude']
        lon = self.aoi_alerts.loc[alert_id]['longitude']
        
        self.map_.center=((lat,lon))
        self.map_.zoom=15
        
    def prev_next_event(self, widget, change, data):
        
        current = self.w_alert_list.v_model
        position = self.w_alert_list.items.index(current)
        last = len(self.w_alert_list.items) - 1
            
        if widget.children==['nxt']:
            if position < last:
                self.w_alert_list.v_model = self.w_alert_list.items[position+1]

        elif widget.children==['prev']:
            if position > 0:
                self.w_alert_list.v_model = self.w_alert_list.items[position-1]
    
        
    def aoi_method_event(self, change):
        
        self.remove_layers()
        
        if change['new'] == 'Select country':
            self.map_.hide_dc()
            su.show_component(self.w_countries)
            
        else:
            su.hide_component(self.w_countries)
            self.map_.show_dc()
            
    
    def add_country_event(self, change):
        
        self.remove_layers()
        
        country_df = COUNTRIES[COUNTRIES['name']==change['new']]
        geometry =  country_df.iloc[0].geometry
        
        lon, lat = [xy[0] for xy in geometry.centroid.xy]
        
        data = eval(country_df.to_json())
        
        aoi = GeoJSON(data=data,
                      name=change['new'], 
                     style={'color': 'green', 'fillOpacity': 0, 'weight': 3})
            
        
        self.aoi = aoi.data['features'][0]['geometry']
        
        min_lon, min_lat, max_lon, max_lat = geometry.bounds

        # Get (x, y) of the 4 cardinal points
        tl = (max_lat, min_lon)
        bl = (min_lat, min_lon)
        tr = (max_lat, max_lon)
        br = (min_lat, max_lon)
        
        self.map_.zoom_bounds([tl,bl, tr, br])
        self.map_.center = (lat, lon)
        self.map_.add_layer(aoi)


    def _serach_items(self):
        
        
        ""
    def validate_inputs(self):
        
        if not self.aoi:
            self.w_alert.add_msg(msg='Please select a valid area of interest...', 
                                 type_='error')
            self.restore_widgets()
            
            raise
    
    def restore_widgets(self):
        
        self.w_run.disabled=False
        self.w_run.loading=False
        self.w_alert_list.items = []
        self.w_alert_list.v_model = None

    def _get_url(self, satellite):
        
        satellites = {
            'viirs': ('SUOMI_VIIRS_C2', 'suomi-npp-viirs-c2'),
            'modis': ('MODIS_C6', 'c6'),
            'viirsnoa': ('J1_VIIRS_C2', 'noaa-20-viirs-c2'),
        
        }
        
        sat = satellites[satellite]
        timespan = self.timespan.replace(' hours', 'h').replace(' days','d')
        
        url=f"https://firms.modaps.eosdis.nasa.gov/data/active_fire/{sat[1]}/csv/{sat[0]}_Global_{timespan}.csv"
        return url
        
    def _get_alerts(self, widget, change, data):
        
        self.validate_inputs()
        widget.toggle_loading()
        
        self.w_alert.add_live_msg(msg='Downloading alerts...', type_='info')

        
        url = self._get_url('viirs')
        confidence=None
        
        df = pd.read_csv(url)
        alerts_gdf = gpd.GeoDataFrame(df, 
                                      geometry=gpd.points_from_xy(df.longitude, 
                                                                  df.latitude), 
                                      crs="EPSG:4326")
        
        if confidence: alerts_gdf = alerts_gdf[alerts_gdf.confidence==confidence]
        
        self.alerts = alerts_gdf
        
        self.aoi_alerts = self._clip_to_aoi()

        self.w_alert.add_msg(msg=f'There are {len(self.aoi_alerts)}' + \
                             f' fire alerts in the last {self.timespan}.', type_='success')
        
        alert_list_item = list(self.aoi_alerts.index)
        self.w_alert_list.items = alert_list_item
        
        json_aoi_alerts = eval(self.aoi_alerts.to_json())
        json_aoi_alerts = GeoJSON(data=json_aoi_alerts,
                                name='Alerts', 
                                point_style={
                                     'radius': 2, 
                                     'color': 
                                     'red', 
                                     'fillOpacity': 0.1, 
                                     'weight': 2
                                 },
                                hover_style={
                                    'color': 'white', 
                                    'dashArray': '0', 
                                    'fillOpacity': 0.5
                                },)
        
        
        self.map_.add_layer(json_aoi_alerts)
        self.w_alerts.disabled = False
        
        
        widget.toggle_loading()
    
    def _clip_to_aoi(self):
        
        # Clip alerts_gdf to the selected aoi
        ""
        self.w_alert.add_live_msg(msg='Clipping alerts to area of interest...', 
                                type_='info')
        
        clip_geometry = shapely.geometry.Polygon(self.aoi['coordinates'][0])
        
        alerts = self.alerts[self.alerts.geometry.intersects(clip_geometry)]
        
        return alerts
    
    def _workspace(self):
        """ Creates the workspace necessary to store and manipulate the module

        return:
            Returns environment Paths

        """

        base_dir = Path(os.path.join('/home', self.USER))

        root_dir = base_dir/'Planet_fire_explorer'
        data_dir = root_dir/'data'

        root_dir.mkdir(parents=True, exist_ok=True)
        data_dir.mkdir(parents=True, exist_ok=True)

        return root_dir, data_dir

In [9]:
ui = UI()

In [10]:
ui

UI(children=[Flex(children=[Card(children=[Flex(children=[TextField(append_icon='mdi-eye-off', class_='mr-2', …

In [11]:
ui.alerts

,latitude,longitude,bright_ti4,scan,track,acq_date,acq_time,satellite,confidence,version,bright_ti5,frp,daynight,geometry
0,-32.93789,18.76212,302.4,0.45,0.39,2021-01-17,0,N,nominal,2.0NRT,290.2,0.7,N,POINT (18.76212 -32.93789)
1,-32.94149,18.76150,300.9,0.45,0.39,2021-01-17,0,N,nominal,2.0NRT,289.8,0.7,N,POINT (18.76150 -32.94149)
2,-32.96203,18.04885,302.8,0.42,0.38,2021-01-17,0,N,nominal,2.0NRT,289.5,1.4,N,POINT (18.04885 -32.96203)
3,-32.96553,18.04819,297.9,0.42,0.38,2021-01-17,0,N,nominal,2.0NRT,287.7,0.6,N,POINT (18.04819 -32.96553)
4,-33.88318,25.62005,301.8,0.34,0.56,2021-01-17,0,N,nominal,2.0NRT,290.2,0.6,N,POINT (25.62005 -33.88318)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106069,-4.71753,-38.88195,342.5,0.53,0.50,2021-01-18,1530,N,nominal,2.0NRT,296.5,9.1,D,POINT (-38.88195 -4.71753)
106070,-4.71917,-38.88472,341.4,0.53,0.50,2021-01-18,1530,N,nominal,2.0NRT,295.2,6.2,D,POINT (-38.88472 -4.71917)
106071,-4.65017,-39.23957,332.9,0.56,0.51,2021-01-18,1530,N,nominal,2.0NRT,277.0,6.5,D,POINT (-39.23957 -4.65017)
106072,-5.04407,-42.49676,326.8,0.47,0.64,2021-01-18,1530,N,nominal,2.0NRT,284.7,3.6,D,POINT (-42.49676 -5.04407)
